# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the previously saved csv file
city_df = pd.read_csv("city_data.csv")

city_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,atuona,84,PF,1587177121,75,-9.80,-139.03,82.35,13.06
1,belaya gora,79,RU,1587176651,89,68.53,146.42,3.60,5.99
2,sanmenxia,100,CN,1587177122,53,34.77,111.18,54.79,13.04
3,nemuro,20,JP,1587176580,26,43.32,145.57,59.00,11.41
4,punta arenas,14,CL,1587176568,75,-53.15,-70.92,44.60,10.29


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Get the key
gmaps.configure(g_key)
# Store latitude and longitude in locations
locations = city_df[['Lat', 'Lng']]
humidity = city_df['Humidity'].astype(float)
# Plot Heatmap and set place
fig = gmaps.figure(zoom_level= 2.1,center = (14,10))
# Create heat layer
heatL = gmaps.heatmap_layer(locations, weights = humidity,
                                 dissipating = False, max_intensity = 400,
                                 point_radius = 3)
#Add layer
fig.add_layer(heatL)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Create DataFrame with specific criteria
new_df = city_df.loc[(city_df['Max Temp'] > 70) &
                     (city_df['Max Temp'] < 80) &
                     (city_df['Cloudiness'] == 0) &
                     (city_df['Wind Speed'] < 10)]
new_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
159,flinders,0,AU,1587176607,42,-34.58,150.86,72.00,5.01
184,saint-leu,0,RE,1587177171,78,-21.15,55.28,71.60,9.17
410,bandipur,0,NP,1587177231,38,27.93,84.42,74.05,1.95
498,mabaruma,0,GY,1587177255,88,8.20,-59.78,74.55,5.39
540,san pedro de macoris,0,DO,1587177264,88,18.46,-69.30,75.99,8.05


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Assign hotel variables
hotel = []
hotel_df = new_df[['City', 'Country', 'Lat', 'Lng']].copy()

# Set parameters and loop through and record hotels
params = {"radius": 5000,
          "types": "lodging",
          "key": g_key}
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    name_address = requests.get(url, params=params).json()
    try:
        hotel.append(name_address['results'][0]['name'])
    except IndexError:
        hotel.append(' ')
        
hotel_df['Hotel Name'] = hotel
hotel_df.dropna()
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
159,flinders,AU,-34.58,150.86,Shellharbour Resort & Conference Centre
184,saint-leu,RE,-21.15,55.28,La Caz Oceane
410,bandipur,NP,27.93,84.42,Bandipur Guest House
498,mabaruma,GY,8.20,-59.78,
540,san pedro de macoris,DO,18.46,-69.30,Apartahotel Guaraguao


In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [7]:
# Add marker layer ont the heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig



Figure(layout=FigureLayout(height='420px'))